In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/data_chg_all.csv')
df.columns

Index(['Chargingevent', 'CPID', 'Connector', 'StartDate', 'StartTime',
       'EndDate', 'EndTime', 'StartDate_num', 'StartTime_num', 'EndDate_num',
       'EndTime_num', 'duration', 'TotalkWh', 'Cost', 'Site', 'Group', 'Model',
       'Model1'],
      dtype='object')

In [4]:
a = pd.read_csv('data/data_chg/data_chg_1.csv')
a.head()

,t,dayofweek,weekend,y_t_1,y
0,2,1,0,0,0
1,3,1,0,0,0
2,4,1,0,0,0
3,5,1,0,0,0
4,6,1,0,0,0


In [13]:
len(a)

8783

In [11]:
a[a.y != 0]

,t,dayofweek,weekend,y_t_1,y
43,45,1,0,0,1
44,46,1,0,1,1
45,47,1,0,1,1
46,48,1,0,1,1
63,65,1,0,0,1
...,...,...,...,...,...
8718,80,5,0,0,1
8719,81,5,0,1,1
8720,82,5,0,1,1
8721,83,5,0,1,1


In [7]:
avg_vhg_occ_rate = pd.read_csv('data/data_chg_pred_occ_t/data_chg_pred_occ_t_1.csv')
avg_vhg_occ_rate.head()

,weekday,weekend
0,0.022222,0.0625
1,0.000000,0.0625
2,0.022222,0.0000
3,0.022222,0.0625
4,0.000000,0.0625


In [3]:
# Convert 'StartDate' to datetime with the correct format
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%m/%d/%Y')

# Define the date range
start_date = pd.Timestamp('2018-03-05')
end_date = pd.Timestamp('2018-06-04')

# Filter the dataset to include only rows within the date range
df = df[(df['StartDate'] >= start_date) & (df['StartDate'] <= end_date)]

len(df)

15901

In [4]:
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
print('Number of days:', len(all_dates))

Number of days: 92


In [5]:
grouped_counts = df.groupby(['Model1'])

print("Charger Counts:")
print(grouped_counts['CPID'].nunique())

Charger Counts:
Model1
Slow     40
fast      8
rapid     9
Name: CPID, dtype: int64


In [6]:
# Count the number of each type of charger
charger_counts = df['Model1'].value_counts()

# Calculate the total number of charging sessions
total_sessions = len(df)

# Calculate the percentage of sessions for each type of charger
charger_percentages = (charger_counts / total_sessions) * 100

print("Charger Percentages:")
print(charger_percentages)

Charger Percentages:
rapid    56.229168
Slow     35.236778
fast      8.534054
Name: Model1, dtype: float64


In [7]:
# Keep only the rapid chargers
rapid_chargers_df = df[df['Model1'] == 'rapid']

# Calculate the median and standard deviation of the 'duration' column for rapid chargers
median_duration = rapid_chargers_df['duration'].median()
std_duration = rapid_chargers_df['duration'].std()

# Define the threshold for outliers
upper_threshold = median_duration + 3 * std_duration
lower_threshold = median_duration - 3 * std_duration

# Filter out the outliers
filtered_df = rapid_chargers_df[(rapid_chargers_df['duration'] >= lower_threshold) & (rapid_chargers_df['duration'] <= upper_threshold )]

# Check the percentage of rows removed
percent_removed = (1 - len(filtered_df) / len(rapid_chargers_df)) * 100
print(f'{percent_removed:.2f}% of outliers were removed.')

0.79% of outliers were removed.


In [8]:
rapid_chargers_df['duration'].describe()

count    8941.000000
mean       28.313723
std        36.412521
min         5.000000
25%        15.000000
50%        24.000000
75%        34.000000
max      1145.000000
Name: duration, dtype: float64

In [9]:
filtered_df['duration'].describe()

count    8870.000000
mean       25.860203
std        14.171344
min         5.000000
25%        15.000000
50%        24.000000
75%        33.000000
max       127.000000
Name: duration, dtype: float64

## **Our Data**

In [10]:
boulder = pd.read_csv('../data/EntireBoulderWithWeather.csv')
boulder.head()

,Unnamed: 0,ObjectId2,Station_Name,Address,City,State_Province,Zip_Postal_Code,Start_Date___Time,Start_Time_Zone,End_Date___Time,...,ObjectID,Date,Year,Month,Day,Maximum T,Minimum T,Precipitation,Snow,Weekday
0,0,1,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,2018-01-01 17:49:00,MDT,2018-01-01 19:52:00,...,0,2018-01-01,2018,January,1,30,12,0.0,0.0,Monday
1,2,3,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,2018-01-02 21:11:00,MDT,2018-01-03 06:23:00,...,2,2018-01-02,2018,January,2,46,12,0.0,0.0,Tuesday
2,3,70196,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,2018-01-02 08:52:00,MDT,2018-01-02 09:16:00,...,1,2018-01-02,2018,January,2,46,12,0.0,0.0,Tuesday
3,6,4,BOULDER / ALPINE ST1,1275 Alpine Ave,Boulder,Colorado,80304,2018-01-03 09:19:00,MDT,2018-01-03 11:14:00,...,3,2018-01-03,2018,January,3,50,20,0.0,0.0,Wednesday
4,7,5,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018-01-03 14:13:00,MDT,2018-01-03 14:30:00,...,4,2018-01-03,2018,January,3,50,20,0.0,0.0,Wednesday


#### Removing Outliers in Energy Consumption Column

In [11]:
# Calculate the median and standard deviation of the `Energy__kWh_` column
median_energy = boulder['Energy__kWh_'].median()
std_energy = boulder['Energy__kWh_'].std()

# Calculate the lower and upper bounds for outliers
lower_bound = median_energy - 3 * std_energy
upper_bound = median_energy + 3 * std_energy

# Filter out the outliers
filtered_boulder = boulder[(boulder['Energy__kWh_'] >= lower_bound) & (boulder['Energy__kWh_'] <= upper_bound)]

# Check the percentage of rows removed
percent_removed = (1 - len(filtered_boulder) / len(boulder)) * 100
print(f'{percent_removed:.2f}% of outliers were removed.')

2.65% of outliers were removed.
